## Setting & Import

In [1]:
import sys
from pathlib import Path

BASE_DIR = Path().resolve()
SUB_FILES_DIR = BASE_DIR / "sub-files"
if str(SUB_FILES_DIR) not in sys.path:
    sys.path.append(str(SUB_FILES_DIR))

from agent_module import run_agent, execution_store

def print_answer(result):
    print(f"✅ 답변: \n{result["answer"]}")
    print("=" * 80)
    # 저장된 결과 조회
    if result["execution_id"]:
        saved = execution_store.get(result["execution_id"])
        print(f"✅ 생성 코드: \n{saved['code']}")
        print("=" * 80)
        print(f"✅ 실행 결과: \n{saved['result']}")

## Test

In [5]:
print("=" * 80)
print("🤖 Agent 대화형 모드")
print("=" * 80)
print("질문을 입력하세요. 종료하려면 '이제 끝!'을 입력하세요.")
print("=" * 80)
print()

# 세션 ID 초기화 (같은 세션을 유지하여 대화 히스토리 연결)
current_session_id = None

while True:
    # 사용자 입력 받기
    question = input("질문: ").strip()
    
    # 종료 조건 확인
    if question == "이제 끝!":
        print()
        print("=" * 80)
        print("👋 대화를 종료합니다. 감사합니다!")
        print("=" * 80)
        break
    
    # 빈 입력 처리
    if not question:
        print("⚠️ 질문을 입력해주세요.\n")
        continue
    
    print()
    print("=" * 80)
    print(f"🔄 처리 중: {question}")
    print("=" * 80)
    print()
    
    try:
        # Agent 실행 (같은 session_id를 사용하여 대화 히스토리 유지)
        result = run_agent(question=question, session_id=current_session_id)
        
        # 첫 번째 질문에서 생성된 session_id를 저장하여 이후 질문에서 재사용
        if current_session_id is None:
            current_session_id = result["session_id"]
            print(f"💡 세션 ID: {current_session_id} (대화 히스토리가 연결됩니다)")
            print()
        
        # 결과 출력
        print_answer(result)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print()
    
    print()
    print("=" * 80)
    print()


🤖 Agent 대화형 모드
질문을 입력하세요. 종료하려면 '이제 끝!'을 입력하세요.


🔄 처리 중: 강남에는 어떤 업종의 공장이 많아?

💡 세션 ID: 048b911d-e34e-4311-a682-ad29129a1eef (대화 히스토리가 연결됩니다)

✅ 답변: 
지금 가지고 있는 공장 데이터에서 “강남”이 포함된 지역(정제_시군구명/정제_시도명 기준)을 추려서 업종별 공장 수를 집계해보면, 상위 업종은 다음과 같습니다.

1. 여자용 겉옷 제조업 – 공장 18개  
2. 경 인쇄업 – 공장 10개  
3. 기타 주변기기 제조업 – 공장 10개  
4. 유선 통신장비 제조업 – 공장 10개  
5. 방송장비 제조업 – 공장 9개  

즉, 이 데이터 기준으로 보면 강남에는  
- 의류(여자용 겉옷),  
- 인쇄업,  
- IT·전자 관련(주변기기, 통신장비, 방송장비)  
같은 업종 공장이 특히 많이 분포해 있습니다.
✅ 생성 코드: 
import pandas as pd

# 강남 지역 필터링: 시군구명 또는 시도명에 '강남'이 포함된 공장만 선택
mask_gangnam = (
    df['정제_시군구명'].fillna('').str.contains('강남', na=False) |
    df['정제_시도명'].fillna('').str.contains('강남', na=False)
)

df_gangnam = df[mask_gangnam].copy()

# 업종(산업분류)별 공장 수 집계
# 공장 수는 공장관리번호의 고유 개수로 계산
agg_df = (
    df_gangnam
    .groupby('정제_대표업종', dropna=True)['공장관리번호']
    .nunique()
    .reset_index(name='공장수')
)

# 업종 코드별 대표 업종명(정제_업종명) 매핑
# 동일 코드에 여러 이름이 있을 수 있으므로, 가장 많이 등장한 이름을 대표로 사용
code_name_map = (
    df_gangnam
    .dropn

## Examples

In [2]:
Q1 = run_agent("서울에서 공장이 가장 많은 구는 어디야?")
print_answer(Q1)

✅ 답변: 
데이터 기준으로 보면,  

- **서울에서 공장이 가장 많은 구는 `금천구`**이고  
- **공장 수는 4,059개**입니다.
✅ 생성 코드: 
import pandas as pd

# 서울 지역만 필터링
seoul_df = df[df['정제_시도명'] == '서울특별시']

# 구(시군구)별 공장 수 집계 (공장관리번호는 고유 공장 수로 distinct count)
factory_count_by_gu = (
    seoul_df
    .groupby('정제_시군구명')['공장관리번호']
    .nunique()
    .reset_index(name='공장수')
    .sort_values('공장수', ascending=False)
    .head(1)
)

return_var = factory_count_by_gu
✅ 실행 결과: 
{'type': 'dataframe', 'shape': [1, 2], 'columns': ['정제_시군구명', '공장수'], 'data': [{'정제_시군구명': '금천구', '공장수': 4059}]}


In [23]:
Q2 = run_agent("지난 20년간 서울에서 공장 수가 가장 많이 증가한 5개 구를 알려줘")
print_answer(Q2)

✅ 답변: 
지난 20년 동안(데이터에서 확인 가능한 최대 연도를 기준으로 직전 20년) 서울에서 공장 수가 가장 많이 증가한 구는 다음 5곳입니다.

1. 금천구: 공장 수 +65개 증가  
2. 영등포구: 공장 수 +10개 증가  
3. 강서구: 공장 수 +8개 증가  
4. 동대문구: 공장 수 +8개 증가  
5. 노원구: 공장 수 +7개 증가  

위 증감 수치는 각 구별로  
- 기간 시작 연도(20년 전)와  
- 기간 종료 연도(가장 최근 연도)  
두 시점의 공장 수(공장관리번호 기준 유니크 수)를 비교해 계산한 값입니다.
✅ 생성 코드: 
import pandas as pd

# 기준 연도 계산 (데이터의 최대 연도를 기준으로 최근 20년)
df_valid_date = df[df['정제_최초등록일'].notna()].copy()
df_valid_date['연도'] = pd.to_datetime(df_valid_date['정제_최초등록일']).dt.year

max_year = df_valid_date['연도'].max()
start_year = max_year - 19

# 서울특별시만 필터링
seoul_df = df_valid_date[df_valid_date['정제_시도명'] == '서울특별시'].copy()

# 분석 기간 내 데이터만 사용
period_df = seoul_df[(seoul_df['연도'] >= start_year) & (seoul_df['연도'] <= max_year)].copy()

# 연도별, 구별 공장 수 (공장관리번호 기준 유니크 카운트)
yearly_counts = (
    period_df
    .groupby(['정제_시군구명', '연도'])['공장관리번호']
    .nunique()
    .reset_index(name='공장수')
)

# 시작 연도와 종료 연도의 공장 수 피벗
yearly_pivot = yearly_counts.pivot(index='정제_시군구명', columns='연도', va

In [24]:
Q3 = run_agent("새로 생긴 공장들이 요즘 어느 지역에 많이 몰려 있어?")
print_answer(Q3)

✅ 답변: 
최근 2년 기준으로 새로 등록된 공장들이 많이 몰려 있는 지역 상위 10곳은 다음과 같습니다. (단위: 공장 수)

1. 서울특별시 금천구 – 420개  
2. 서울특별시 구로구 – 135개  
3. 서울특별시 영등포구 – 56개  
4. 서울특별시 성동구 – 49개  
5. 서울특별시 강서구 – 39개  
6. 서울특별시 중랑구 – 33개  
7. 서울특별시 중구 – 30개  
8. 서울특별시 송파구 – 26개  
9. 서울특별시 노원구 – 21개  
10. 서울특별시 동대문구 – 19개  

요약하면, 최근에는 서울 안에서도 특히 **금천구와 구로구**에 새 공장이 매우 많이 집중되어 있고, 그 외에도 **영등포구·성동구·강서구** 등 서울 서남·동북권 일부 지역에 공장 설립이 상대적으로 활발한 편입니다.
✅ 생성 코드: 
import pandas as pd

# 기준 날짜 컬럼: '정제_최초등록일' 사용 (YYYY-MM-DD 형식)
# 최근 2년 내 설립된 공장만 필터링

# 날짜 컬럼을 datetime으로 변환
_df = df.copy()
_df['정제_최초등록일'] = pd.to_datetime(_df['정제_최초등록일'], errors='coerce')

# 기준일: 데이터 내 최대 등록일 기준으로 최근 2년
max_date = _df['정제_최초등록일'].max()
cutoff_date = max_date - pd.DateOffset(years=2)

recent_df = _df[_df['정제_최초등록일'] >= cutoff_date].copy()

# 지역 기준: 시도/시군구 조합으로 지역명 생성
recent_df['지역'] = (
    recent_df['정제_시도명'].fillna('')
    + ' '
    + recent_df['정제_시군구명'].fillna('')
).str.strip()

# 공장 수는 공장관리번호의 고유 개수로 집계
region_counts = (
    recent_df
    .dropna(sub

In [25]:
Q4 = run_agent("금천구에서 규모가 가장 회사는 어디야? 종업원수, 규모를 종합적으로 고려해서 알려줘")
print_answer(Q4)

✅ 답변: 
지금 가지고 있는 데이터 기준으로,  
서울 금천구에서 **규모(종업원수·매출/자산 등 종합점수)가 가장 큰 회사 상위 5곳**은 다음과 같습니다.

1. **롯데이노베이트(주)아이씨씨**  
   - 종합점수: **48.29** (표준화된 종업원수·규모지표를 합산한 점수, 가장 높음)  
   - 종업원수: **2,184명**  
   → 금천구 내에서 종업원 규모와 재무 규모를 종합했을 때 **가장 큰 회사**로 나타납니다.

2. **(주)신영와코루**  
   - 종합점수: **14.39**  
   - 종업원수: **660명**

3. **(주)엠씨넥스**  
   - 종합점수: **10.25**  
   - 종업원수: **474명**

4. **(주)제이시스메디칼**  
   - 종합점수: **7.94**  
   - 종업원수: **370명**

5. **(주)로옴코리아**  
   - 종합점수: **6.89**  
   - 종업원수: **323명**

### 어떻게 계산했는지 요약
- **금천구 소재** 회사만 필터링(‘정제_시군구명’에 ‘금천구’ 포함).
- **종업원수(종업원합계)**와 **매출액 또는 자산총액**을 사용해 각각 Z-score(표준점수)로 변환.
- 표준화된 값들을 **합산한 ‘종합점수’**를 만들고,  
  회사명 단위로 평균을 내서 **종합점수가 가장 높은 순으로 정렬**했습니다.

따라서, 질문하신 “금천구에서 규모가 가장 큰 회사”는  
이 데이터 기준으로는 **롯데이노베이트(주)아이씨씨**라고 볼 수 있습니다.
✅ 생성 코드: 
import numpy as np

# 1. 금천구 소재 회사 필터링 (소재지 컬럼명 가정: '정제_시군구명')
mask_geumcheon = df['정제_시군구명'].astype(str).str.contains('금천구', na=False)
df_gc = df.loc[mask_geumcheon].copy()

# 2. 사용할 규모 지표 컬럼 결정 (매출액 vs 자산총액)
#    

In [3]:
Q5 = run_agent("공장의 면적 대비 직원 수가 많은 공장 상위 5개를 알려줘")
print_answer(Q5)

✅ 답변: 
공장 면적 대비 직원 수(인/㎡)가 가장 높은 상위 5개 공장은 다음과 같습니다.

1. 공장관리번호: 115452016367230  
   - 회사명: (주)알바이오  
   - 위치: 서울특별시 금천구  
   - 용지면적: 24.60 ㎡  
   - 종업원합계: 171명  
   - 면적 대비 직원 수: 약 6.95 인/㎡  

2. 공장관리번호: 115302002044635  
   - 회사명: (주)배산엔지니어링  
   - 위치: 서울특별시 구로구  
   - 용지면적: 26.76 ㎡  
   - 종업원합계: 162명  
   - 면적 대비 직원 수: 약 6.05 인/㎡  

3. 공장관리번호: 115602018420131  
   - 회사명: (주)브리지텍  
   - 위치: 서울특별시 영등포구  
   - 용지면적: 119.88 ㎡  
   - 종업원합계: 540명  
   - 면적 대비 직원 수: 약 4.50 인/㎡  

4. 공장관리번호: 115452009183019  
   - 회사명: (주)큐디스  
   - 위치: 서울특별시 금천구  
   - 용지면적: 22.74 ㎡  
   - 종업원합계: 96명  
   - 면적 대비 직원 수: 약 4.22 인/㎡  

5. 공장관리번호: 115452005104635  
   - 회사명: 엠텍비젼(주)  
   - 위치: 서울특별시 금천구  
   - 용지면적: 198.57 ㎡  
   - 종업원합계: 750명  
   - 면적 대비 직원 수: 약 3.78 인/㎡  

위 순위는 각 공장별로 용지면적과 종업원합계를 합산한 뒤, `종업원합계 ÷ 용지면적` 값이 큰 순서대로 정렬해 상위 5개를 추출한 결과입니다.
✅ 생성 코드: 
import pandas as pd

# 직원 수와 공장 면적 대비 직원 수 계산
# 면적은 '용지면적'을 사용하고, 직원 수는 '종업원합계' 사용
# 공장관리번호 기준으로 집계 후 비율 계산

# 유효한 면적과 직원 수만 필터링
_df = df.copy()
_d